<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_DTS201_Introduction_to_Data_Science/blob/main/notebooks/DTS201_Week6_PH_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DTS 201 – INTRODUCTION TO DATA SCIENCE  
## WEEK 6 PRACTICAL (PH): DATA MINING – MARKET BASKET ANALYSIS & CLUSTERING

**Instructor:** DR SAKINAT FOLORRUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA  
**Department:** COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO-IWOYE, OGUN STATE, NIGERIA  

---

### 🎯 Practical Learning Objectives (Week 6)

By the end of this practical session, you should be able to:

- Explain the basic ideas of **data mining**, **market basket analysis**, and **clustering**.  
- Represent **transactional data** (e.g. shopping baskets) in a form suitable for analysis.  
- Use **frequent itemset mining** (Apriori) to discover common item combinations.  
- Derive simple **association rules** with support, confidence, and lift.  
- Perform **K-Means clustering** on simple feature data.  
- Interpret clusters and rules in plain language for decision-making.

> 💡 This notebook uses Python instead of R, but implements the same ideas in your course outline: *“Market basket analysis and clustering on transactional datasets”*.


## 1. Getting Started: What Is Data Mining?

**Data mining** is the process of discovering **patterns**, **relationships**, and **structures** in large datasets.

In this lab, we focus on two classic tasks:

1. **Market Basket Analysis (Association Rules)**  
   - Find which items are frequently bought together.  
   - Useful in retail, e-commerce, recommender systems.

2. **Clustering**  
   - Group similar records (e.g. customers with similar behaviour).  
   - Useful for segmentation, targeting, and profiling.

You will work with a simple **transactional dataset**, where each transaction represents items bought together.


### 1.1 Optional: Mount Google Drive (If Using Google Colab)

In [ ]:
# Run this cell ONLY if:
# 1. You are working in Google Colab, AND
# 2. Your transactional dataset is stored in Google Drive.

# Import the drive helper from google.colab to mount Google Drive
from google.colab import drive  # Allows Colab to access files in your Google Drive

# Mount your Google Drive
drive.mount('/content/drive')  # Follow the on-screen instructions for authorization

# After mounting, you can access files under '/content/drive/MyDrive/'
# Example: data_path = '/content/drive/MyDrive/Datasets/transactions.csv'

## 2. Importing Required Libraries

In [ ]:
# Import pandas for working with tabular and transactional data
import pandas as pd

# Import numpy for numerical computations
import numpy as np

# Import matplotlib for creating charts and plots
import matplotlib.pyplot as plt

# Import seaborn for more attractive and statistical visualizations
import seaborn as sns

# Import KMeans for clustering from scikit-learn
from sklearn.cluster import KMeans

# Try to import Apriori and association_rules from mlxtend (if installed)
# These are commonly used for market basket analysis in Python
try:
    from mlxtend.frequent_patterns import apriori, association_rules  # For frequent itemsets and association rules
    MLXTEND_AVAILABLE = True  # Flag to indicate mlxtend is available
except ImportError:
    MLXTEND_AVAILABLE = False  # If mlxtend is not installed, we will note this later

# Ensure plots are displayed inside this notebook
%matplotlib inline

# Set a consistent visual style for all plots
plt.style.use("seaborn-v0_8")

> ⚠️ **Note for Students:**  
> If `mlxtend` is not installed in your environment, you may need to run:  
> `!pip install mlxtend` in a separate cell (especially in Google Colab).  
> Ask your instructor or lab assistant if you are unsure.


## 3. Loading the Transactional Dataset

> 🔧 **Dataset Format (Example):**  
> A simple CSV file where each row is an item in a transaction, for example:

| TransactionID | Item       |
|---------------|-----------|
| 1             | Bread     |
| 1             | Butter    |
| 1             | Milk      |
| 2             | Bread     |
| 2             | Eggs      |
| 3             | Milk      |

> Several rows may share the same `TransactionID` but with different items.


In [ ]:
# Specify the path to your transactional dataset
# Example (local): data_path = "transactions.csv"
# Example (Colab + Drive): data_path = "/content/drive/MyDrive/Datasets/transactions.csv"
data_path = "transactions.csv"  # TODO: change this to your actual transactional data file

# Load the CSV into a pandas DataFrame
transactions = pd.read_csv(data_path)  # Reads the CSV file into 'transactions'

# Display the first few rows to understand the structure
transactions.head()  # Shows sample transactions with TransactionID and Item

> ✏️ **Student Task 1:**  
> - What does one row in the `transactions` dataset represent?  
> - Identify the column that represents the **transaction/basket ID** and the column that represents the **item**.  
> - Are there any additional useful columns (e.g. customer ID, quantity, date)?


### 3.1 Basic Inspection of the Transactional Data

In [ ]:
# Check the shape of the dataset (rows, columns)
transactions.shape  # Helps you see how many records and fields you have

In [ ]:
# Display summary information about the dataset
transactions.info()  # Shows column names, data types, and non-null counts

In [ ]:
# Check how many unique transactions and items are present
n_transactions = transactions['TransactionID'].nunique()  # Number of distinct transaction IDs
n_items = transactions['Item'].nunique()                  # Number of distinct items

print("Number of unique transactions:", n_transactions)
print("Number of unique items:", n_items)

## 4. Preparing Data for Market Basket Analysis

For market basket analysis, we often convert the data into a **basket–item matrix**, where:

- Each **row** is a transaction (basket).  
- Each **column** is an item (e.g. Bread, Milk).  
- Each cell is 1 if the item appears in the transaction, otherwise 0.

This is sometimes called a **one-hot encoded** or **indicator** matrix.


In [ ]:
# Create a basket-item matrix using pivot_table
# Step 1: Create a cross-tabulation of TransactionID vs Item
basket = transactions.pivot_table(
    index='TransactionID',      # Rows are transaction IDs
    columns='Item',             # Columns are individual items
    aggfunc=lambda x: 1,        # Any occurrence counts as 1
    fill_value=0                # Missing combinations are filled with 0
)

# Display the first few rows of the basket matrix
basket.head()  # Each row is a transaction, each column an item, values are 0/1

> ✏️ **Student Task 2:**  
> - Count how many items are represented as columns in `basket`.  
> - Is this number small (e.g. ≤ 20) or large? How might a very large number of items affect computation?


## 5. Discovering Frequent Itemsets (Apriori Algorithm)

We now find **frequent itemsets** – combinations of items that occur together in many transactions.

Key concept:

- **Support** of an itemset = (Number of transactions containing the itemset) / (Total number of transactions).


In [ ]:
# Check whether mlxtend is available before running Apriori
if not MLXTEND_AVAILABLE:
    print("mlxtend is not installed. Please run '!pip install mlxtend' in a new cell and re-run this section.")
else:
    # Use the apriori function to find frequent itemsets
    # min_support controls the minimum fraction of transactions that must contain the itemset
    frequent_itemsets = apriori(
        basket,                # Basket-item matrix with 0/1 values
        min_support=0.1,       # TODO: adjust this threshold depending on your dataset (e.g., 0.05, 0.2)
        use_colnames=True      # Use the actual item names instead of column indices
    )

    # Sort itemsets by support (most frequent first)
    frequent_itemsets = frequent_itemsets.sort_values(by="support", ascending=False)

    # Display the top frequent itemsets
    frequent_itemsets.head()

> ✏️ **Student Task 3:**  
> - Increase or decrease `min_support` and observe how the number of frequent itemsets changes.  
> - What happens if `min_support` is very high? What happens if it is very low?


## 6. Generating Association Rules

From frequent itemsets, we can derive **association rules** of the form:

\[ X \Rightarrow Y \]

Examples:

- {Bread} → {Butter}  
- {Diapers, Beer} → {Chips}

Important metrics:

- **Support** – how often the full itemset (X ∪ Y) appears.  
- **Confidence** – P(Y | X): how often Y appears when X is present.  
- **Lift** – how many times more often X and Y occur together than expected if they were independent.


In [ ]:
if not MLXTEND_AVAILABLE:
    print("mlxtend is not installed. Please run '!pip install mlxtend' in a new cell and re-run this section.")
else:
    # Generate association rules from the frequent itemsets
    rules = association_rules(
        frequent_itemsets,      # Frequent itemsets discovered by Apriori
        metric="confidence",    # Use confidence as the primary metric
        min_threshold=0.5       # Minimum confidence threshold
    )

    # Sort rules by confidence (highest first)
    rules = rules.sort_values(by="confidence", ascending=False)

    # Display the top 10 rules
    rules.head(10)

> ✏️ **Student Task 4:**  
> - Look at the top 5 rules. For each rule, rewrite it in **plain English** (e.g. “If a customer buys X, they are likely to also buy Y”).  
> - Pay attention to **support**, **confidence**, and **lift** – which rules seem most interesting for a shop manager?


## 7. Clustering Customers (K-Means)

Now we turn to **clustering**, another important data mining task.

We will treat each transaction (or customer) as a point in a feature space and use **K-Means** to group similar ones.

For simplicity, we can:

- Use the same **basket matrix**, where each row is a transaction and columns are items (0/1).  
- Or aggregate by **CustomerID** if the dataset has that column.

Here, we cluster transactions based on which items appear in them.


In [ ]:
# For clustering, we will reuse the 'basket' matrix
# Each transaction is represented by a vector of 0s and 1s for each item

# Decide how many clusters (groups) we want
k = 3  # TODO: you can try different values such as 2, 3, 4, 5

# Create an instance of the KMeans algorithm
kmeans = KMeans(n_clusters=k, random_state=42)  # random_state for reproducibility

# Fit the clustering model to the basket data
kmeans.fit(basket)  # The model learns k cluster centers based on item patterns

# Get the cluster labels assigned to each transaction
cluster_labels = kmeans.labels_  # Each transaction gets a cluster index (0, 1, ..., k-1)

# Add the cluster labels as a new column in the basket DataFrame
basket_with_clusters = basket.copy()
basket_with_clusters['Cluster'] = cluster_labels

# Display the first few rows to see transactions and their assigned clusters
basket_with_clusters.head()

> ✏️ **Student Task 5:**  
> - Count how many transactions fall into each cluster. Are the clusters roughly balanced or very uneven?  
> - Try changing `k` from 3 to 2 or 4 and re-run the clustering. How does the distribution of cluster sizes change?


### 7.1 Interpreting Cluster Profiles

In [ ]:
# Compute the average presence of each item per cluster
# This shows, for each cluster, the proportion of transactions that contain a given item
cluster_profiles = basket_with_clusters.groupby('Cluster').mean()

# Display the cluster profiles (average item presence per cluster)
cluster_profiles

> ✏️ **Student Task 6:**  
> - Look at `cluster_profiles`. For each cluster, identify 1–3 items with the **highest average values** (closest to 1).  
> - In plain language, describe each cluster. For example:  
>   - “Cluster 0: transactions mainly containing bread, eggs, and milk.”  
>   - “Cluster 1: transactions focused on soft drinks and snacks.”


## 8. (Optional) Visualizing Clusters in 2D

Basket data may have many dimensions (one per item), so visualizing clusters directly is difficult.

Optionally, we can use **dimensionality reduction** (e.g. PCA) to project transactions into 2D for plotting.

> This section is optional and more advanced – try it if you are curious.


In [ ]:
from sklearn.decomposition import PCA  # Import PCA for dimensionality reduction

# Initialize PCA to reduce data to 2 dimensions
pca = PCA(n_components=2, random_state=42)

# Apply PCA on the basket-item matrix (excluding the 'Cluster' column)
basket_items_only = basket_with_clusters.drop(columns=['Cluster'])  # Drop cluster labels for PCA input
basket_2d = pca.fit_transform(basket_items_only)  # Transform high-dimensional data to 2D

# Create a DataFrame with the 2D coordinates and cluster labels
basket_2d_df = pd.DataFrame(basket_2d, columns=['PC1', 'PC2'])  # PC1 and PC2 are the two principal components
basket_2d_df['Cluster'] = cluster_labels  # Add cluster labels to the DataFrame

# Plot the 2D projection with cluster coloring
plt.figure(figsize=(7, 5))

# Use seaborn scatterplot to visualize the 2D points colored by cluster
sns.scatterplot(
    data=basket_2d_df,
    x='PC1',
    y='PC2',
    hue='Cluster',
    palette='tab10'
)

# Add title and labels
plt.title("2D Visualization of Transaction Clusters (PCA Projection)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")

# Display the legend and plot
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()

## 9. Mini Data Mining Exercise

> 🧪 **Student Activity:**  
> Prepare a short **mini-report** (bullet points or a short paragraph) based on your analysis:

1. Describe your transactional dataset (source, number of transactions, examples of items).  
2. Summarize **2–3 interesting association rules** and what they mean for a store or organisation.  
3. Describe each cluster in 1–2 sentences (what makes it unique?).  
4. Suggest **one practical decision** that could be made based on your findings  
   (e.g. shelf arrangement, promotions, cross-selling strategies).  


## 10. Summary and Reflection (Week 6 PH)

In this Week 6 practical, you have:

- Represented transactional data in a basket–item matrix.  
- Applied **Apriori** to discover frequent itemsets.  
- Generated **association rules** and interpreted support, confidence, and lift.  
- Used **K-Means clustering** to group similar transactions based on item patterns.  
- Interpreted clusters and rules to support decision-making.

> 🧠 **Final Reflection (write 1–2 paragraphs):**  
> - Which technique did you find more intuitive: association rules or clustering? Why?  
> - How could market basket analysis and clustering be applied in Nigerian contexts (e.g. supermarkets, online stores, mobile money, local markets)?  
> - What additional data would make your analysis more powerful or realistic?
